In [14]:
import pandas as pd

from datetime import datetime

from user_type_utils import user_type_utils

In [15]:
course_info_df = pd.read_csv('data/raw/365_course_info.csv')
course_ratings_df = pd.read_csv('data/raw/365_course_ratings.csv')
student_info_df = pd.read_csv('data/raw/365_student_info.csv')
student_learning_df = pd.read_csv('data/raw/365_student_learning.csv')
student_engagement_df = pd.read_csv('data/raw/365_student_engagement.csv')
student_purchases_df = pd.read_csv('data/raw/365_student_purchases.csv')

In [16]:
course_ratings_df.date_rated = pd.to_datetime(course_ratings_df.date_rated)
student_info_df.date_registered = pd.to_datetime(student_info_df.date_registered)
student_learning_df.date_watched = pd.to_datetime(student_learning_df.date_watched)
student_engagement_df.date_engaged = pd.to_datetime(student_engagement_df.date_engaged)
student_purchases_df.date_purchased = pd.to_datetime(student_purchases_df.date_purchased)

#### A funnel showing the total number of users from a given country:
- Display the first 5 countries with the largest number of users.
- Depict each country as a horizontal bar.
- The bar length depends on the number of students from that country.
- Stack all 5 horizontal bars and sort the chart in descending order.

In [6]:
merged_student_info_purchase = pd.merge(student_info_df, student_purchases_df, on='student_id', how='left')
merged_student_info_purchase = merged_student_info_purchase.drop_duplicates(subset='student_id')

utils = user_type_utils(merged_student_info_purchase)

merged_student_info_purchase['paid'] = merged_student_info_purchase.student_id.map(utils.is_paid_tier)
merged_student_info_purchase.purchase_type = merged_student_info_purchase.apply(lambda cols : utils.replace_purchase_type(cols.paid, cols.purchase_type), axis=1)

merged_student_info_purchase = merged_student_info_purchase.drop(['date_purchased', 'purchase_id'], axis=1)

# merged_student_info_purchase.to_csv('data/processed/merged_student_info_purchase.csv', index=False)

In [17]:
merged_student_info_purchase = pd.read_csv('data/processed/merged_student_info_purchase.csv')

In [18]:
student_country = merged_student_info_purchase.copy()
student_country = student_country[student_country.purchase_type == student_country.purchase_type]
student_country = student_country[['student_country']].groupby('student_country').size().reset_index().sort_values(by=0, ascending=False).rename(columns={0: 'students'})

student_country.head()

,student_country,students
75,IN,6933
167,US,4768
49,EG,3003
56,GB,1748
119,NG,1718


#### A funnel showing the minutes watched on the platform by users from a given country:
- Display the first 5 countries with the largest number of users.
- Depict each country as a horizontal bar.
- The length of the bar depends on the minutes watched by each country.
- Stack all 5 horizontal bars and sort the chart in descending order according to the number of users.

In [19]:
# country_minutes_watched = pd.merge(merged_student_info_purchase, student_learning_df[['student_id', 'minutes_watched']], on='student_id', how='left')
# country_minutes_watched = country_minutes_watched[['student_country', 'minutes_watched']].groupby('student_country').sum().reset_index().sort_values(by='minutes_watched', ascending=False)

# platform = pd.merge(student_country, country_minutes_watched, on='student_country')
# platform.head()


purchase_type = 'Quarterly'

if purchase_type != 'All':
    student_country = merged_student_info_purchase.copy()
    student_country = student_country[student_country.purchase_type == purchase_type]
    student_country = student_country[['student_country']].groupby('student_country').size().reset_index().sort_values(by=0, ascending=False).rename(columns={0: 'students'})

    country_minutes_watched = pd.merge(merged_student_info_purchase, student_learning_df[['student_id', 'minutes_watched']], on='student_id', how='left')
    country_minutes_watched = country_minutes_watched[country_minutes_watched.purchase_type == purchase_type]
    country_minutes_watched = country_minutes_watched[['student_country', 'minutes_watched']].groupby('student_country').sum().reset_index().sort_values(by='minutes_watched', ascending=False)

    platform = pd.merge(student_country, country_minutes_watched, on='student_country')

platform.head()

,student_country,students,minutes_watched
0,AE,2,1237.8
1,US,2,602.8
2,CA,1,285.0
3,GB,1,1506.2
4,IN,1,652.8


student_country	students	minutes_watched
0	IN	6933	183259.0
1	US	4768	449669.9
2	EG	3003	66529.3
3	GB	1748	113305.7
4	NG	1718	75111.9

#### A bar-and-line chart showing the minutes watched:
- The height of the bars depends on the number of minutes watched.
- The line represents the average number of minutes watched.
- Visualize it monthly.

In [12]:
monthly_average_minutes_watched = pd.merge(merged_student_info_purchase, student_learning_df, on='student_id', how='left')
# monthly_average_minutes_watched = monthly_average_minutes_watched[monthly_average_minutes_watched.paid == False]
monthly_average_minutes_watched['month'] = monthly_average_minutes_watched.date_watched.apply(lambda date : date.month)
monthly_average_minutes_watched['average_minutes_watched'] = monthly_average_minutes_watched.minutes_watched
monthly_average_minutes_watched = monthly_average_minutes_watched[['month', 'minutes_watched', 'average_minutes_watched']].groupby('month').agg({'minutes_watched': 'sum', 'average_minutes_watched': 'mean'}).reset_index().round(2)

monthly_average_minutes_watched

,month,minutes_watched,average_minutes_watched
0,1.0,86953.2,24.45
1,2.0,129973.1,28.34
2,3.0,179715.1,29.68
3,4.0,173665.8,29.91
4,5.0,161828.4,28.27
5,6.0,192599.5,27.86
6,7.0,203366.5,29.30
7,8.0,323322.7,31.90
8,9.0,204191.5,25.75
9,10.0,186561.1,27.09


In [190]:
monthly_average_minutes_watched = student_learning_df.copy()

monthly_average_minutes_watched['month'] = monthly_average_minutes_watched.date_watched.apply(lambda date : date.month)
monthly_average_minutes_watched['average_minutes_watched'] = monthly_average_minutes_watched.minutes_watched
monthly_average_minutes_watched = monthly_average_minutes_watched[['month', 'minutes_watched', 'average_minutes_watched']].groupby('month').agg({'minutes_watched': 'sum', 'average_minutes_watched': 'mean'}).reset_index().round(2)

monthly_average_minutes_watched

,month,minutes_watched,average_minutes_watched
0,1,86953.2,24.45
1,2,129973.1,28.34
2,3,179715.1,29.68
3,4,173665.8,29.91
4,5,161828.4,28.27
5,6,192599.5,27.86
6,7,203366.5,29.30
7,8,323322.7,31.90
8,9,204191.5,25.75
9,10,186561.1,27.09


#### A bar chart showing the number of registered users:
- The height of the bar represents the number of newly registered users.
- A number of the students in a given bar have also onboarded (following the definition of an onboarded student). These are to be colored differently so we can visually assess how this number - compares to the total number of registered users.
- Visualize it monthly.

In [29]:
monthly_registered_students = pd.merge(merged_student_info_purchase, student_engagement_df, on='student_id', how='left')
monthly_registered_students.date_registered = pd.to_datetime(monthly_registered_students.date_registered)
monthly_onboarded_students = monthly_registered_students.copy()

monthly_registered_students = monthly_registered_students.drop_duplicates(subset='student_id')

# monthly_registered_students = monthly_registered_students[monthly_registered_students.purchase_type == 'Quarterly']

monthly_registered_students['month'] = monthly_registered_students.date_registered.apply(lambda date : date.month)
monthly_registered_students = monthly_registered_students[['month']].groupby(['month']).size().reset_index()
monthly_registered_students = monthly_registered_students.rename(columns={0: 'students'})

# monthly_onboarded_students = monthly_onboarded_students[monthly_onboarded_students.purchase_type == 'Quarterly']

monthly_onboarded_students['month'] = monthly_onboarded_students.date_engaged.apply(lambda date : date.month)
monthly_onboarded_students = monthly_onboarded_students[['month']].groupby(['month']).size().reset_index()
monthly_onboarded_students = monthly_onboarded_students.rename(columns={0: 'onboarded'})

students_registered_onboarded = pd.merge(monthly_registered_students, monthly_onboarded_students, on='month')
students_registered_onboarded

,month,students,onboarded
0,1,3312,3588
1,2,3173,4632
2,3,3591,6118
3,4,3187,5881
4,5,2985,5774
5,6,4152,7007
6,7,3306,7045
7,8,5523,10347
8,9,3370,8028
9,10,2631,6951


In [7]:
monthly_registered_students = student_info_df.copy()
monthly_onboarded_students = student_engagement_df.copy()

monthly_registered_students['month'] = monthly_registered_students.date_registered.apply(lambda date : date.month)
monthly_registered_students = monthly_registered_students[['month']].groupby('month').size().reset_index()
monthly_registered_students = monthly_registered_students.rename(columns={0: 'students'})

monthly_onboarded_students['month'] = monthly_onboarded_students.date_engaged.apply(lambda date : date.month)
monthly_onboarded_students = monthly_onboarded_students[['month']].groupby('month').size().reset_index()
monthly_onboarded_students = monthly_onboarded_students.rename(columns={0: 'onboarded'})

students_registered_onboarded = pd.merge(monthly_registered_students, monthly_onboarded_students, on='month')

,month,students,onboarded
0,1,3312,3588
1,2,3173,4632
2,3,3591,6118
3,4,3187,5881
4,5,2985,5774
5,6,4152,7007
6,7,3306,7045
7,8,5523,10347
8,9,3370,8028
9,10,2631,6951


#### A table with 5 columns showing the top 5 most watched courses:
- The first column shows the courses’ name.
- The second column shows the total number of minutes watched from each course.
- The third column shows the average minutes watched (number of minutes divided by the unique number of users that have watched the course).
- The fourth column shows the number of ratings for each of these courses.
- The fifth and final column shows the average rating for each course.

In [18]:
course_info_ratings = pd.merge(course_info_df, course_ratings_df[['course_id', 'course_rating']], on='course_id').groupby(['course_id', 'course_title']).mean().reset_index().round(2)

course_minutes_watched = pd.merge(course_info_ratings, student_learning_df[['course_id', 'minutes_watched', 'student_id']], on='course_id')
course_minutes_watched = course_minutes_watched.groupby(['course_id', 'course_title', 'course_rating']).agg({'minutes_watched': 'sum', 'student_id': 'size'}).reset_index()
course_minutes_watched['mean_minutes_watched'] = (course_minutes_watched.minutes_watched / course_minutes_watched.student_id).round(2)
course_minutes_watched = course_minutes_watched.drop('student_id', axis=1)

course_minutes_watched = course_minutes_watched.sort_values(by='minutes_watched', ascending=False)

course_minutes_watched.head()

,course_id,course_title,course_rating,minutes_watched,mean_minutes_watched
4,7,Introduction to Data and Data Science,4.86,333265.9,21.71
8,14,SQL,4.82,234824.8,35.04
13,21,Statistics,4.81,193279.6,27.43
15,23,Introduction to Excel,4.78,164472.1,27.27
10,16,Python Programmer Bootcamp,4.82,155164.3,33.53


In [19]:
course_info_ratings = pd.merge(course_info_df, course_ratings_df[['course_id', 'course_rating']], on='course_id')
course_info_ratings = course_info_ratings.rename(columns={'course_rating': 'average_course_rating'})
course_info_ratings['ratings'] = course_info_ratings.average_course_rating
course_info_ratings = course_info_ratings.groupby(['course_id', 'course_title']).agg({'average_course_rating': 'mean', 'ratings': 'size'}).reset_index().round(2)

course_minutes_watched = pd.merge(course_info_ratings, student_learning_df[['course_id', 'minutes_watched']], on='course_id')
course_minutes_watched['average_minutes_watched'] = course_minutes_watched.minutes_watched
course_minutes_watched = course_minutes_watched.groupby(['course_id', 'course_title', 'average_course_rating', 'ratings']).agg({'minutes_watched': 'sum', 'average_minutes_watched': 'mean'}).reset_index().round(2)
course_minutes_watched = course_minutes_watched.sort_values(by='minutes_watched', ascending=False)

course_minutes_watched.head()

,course_id,course_title,average_course_rating,ratings,minutes_watched,average_minutes_watched
4,7,Introduction to Data and Data Science,4.86,572,333265.9,21.71
8,14,SQL,4.82,125,234824.8,35.04
13,21,Statistics,4.81,215,193279.6,27.43
15,23,Introduction to Excel,4.78,242,164472.1,27.27
10,16,Python Programmer Bootcamp,4.82,101,155164.3,33.53
